In [1]:
from Nexullance_IT import Nexullance_IT
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from topologies.DDF import DDFtopo
import globals as gl
import numpy as np
import copy

In [2]:
config=(36, 5)
EPR=3
ddf_network = DDFtopo(config[0], config[1])

In [3]:
Cap_remote = 10 #GBps
Cap_local = 10 #GBps
M_EPs = gl.generate_uniform_traffic_pattern(config[0], EPR) # an numpy matrix filled with ones
scaling_factor = 1
M_EPs = scaling_factor * M_EPs
M_R = gl.convert_M_EPs_to_M_R(M_EPs, config[0], EPR)

In [4]:
ASP = ddf_network.calculate_all_shortest_paths()
ECMP_ASP = gl.ECMP(ASP)
remote_link_flows, local_link_flows = ddf_network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EPs)
max_remote_link_load = np.max(remote_link_flows)/Cap_remote
max_local_link_load = np.max(local_link_flows)/Cap_local
print("Max remote link load: ", max_remote_link_load)
print("Max local link load: ", max_local_link_load)
if max_remote_link_load > 1.0 or max_local_link_load > 1.0:
    print("network congestion may present")
    mu = max_local_link_load/max_remote_link_load
    if mu > 1.0:
        print(f"mu = {mu} > 1, max local link load is higher")
    else:
        print(f"mu = {mu} < 1, max remote link load is higher")
else:
    print("network is probably not congested")
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, max_remote_link_load, max_local_link_load), "GBps")


Max remote link load:  20.775
Max local link load:  10.7
network congestion may present
mu = 0.5150421179302046 < 1, max remote link load is higher
network_total_throughput =  556.245487364621 GBps


In [5]:
nexu_it = Nexullance_IT(ddf_network.nx_graph, M_R, Cap_remote)

In [6]:
nexu_it.initialize()

# test method 1

In [7]:
nexu_it_test = copy.deepcopy(nexu_it)
nexu_it_test.optimization_method_1(True)

20.77500000000001

In [8]:
def test_weight_function( s: int, d: int, edge_attributes: dict):
    # define the weight function for the dijkstra or the bellman-ford algorithm.
    alpha: float = 10.0
    beta: float = 1.0
    return alpha+edge_attributes['load']**beta

nexu_it_test.optimization_method_1(True, test_weight_function)

#TODO: method1 does not converge, need to tune the parameters of weights?

17.1

# test method 2

In [9]:
nexu_it_test_after_1 = copy.deepcopy(nexu_it_test)

def test_weight_function( s: int, d: int, edge_attributes: dict):
    # define the weight function for the dijkstra or the bellman-ford algorithm.
    alpha: float = 10.0
    beta: float = 20.0
    return alpha+edge_attributes['load']**beta

In [10]:
num_method_2 = 10
step = 0.5
results_method_2 = {}
for i in range(num_method_2):
    max_link_loads = nexu_it_test_after_1.optimization_method_2(step, test_weight_function)
    results_method_2[step] = max_link_loads
    step *= 0.5

In [11]:
results_method_2

{0.5: (True,
  [17.1,
   17.1,
   17.1,
   17.1,
   16.650000000000002,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003]),
 0.25: (True,
  [16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003]),
 0.125: (True,
  [16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   16.200000000000003,
   15.975000000000003,
   15.975000000000003,
   15.862500000000011,
   15.975000000000001]),
 0.0625: (True,
  [15.975000000000001,
   15.862500000000002,
   15.862500000000002,
   15.975000000000003,
   15.806250000000004,
   15.750000000000007,
   15.806250000000004,
   15.750000000000005,
   15.80625,
   15.80625,


# test optimize()

In [12]:
nexu_it = Nexullance_IT(ddf_network.nx_graph, M_R, Cap_remote)

def method_1_weights( s: int, d: int, edge_attributes: dict):
    # define the weight function for the dijkstra or the bellman-ford algorithm.
    alpha: float = 10.0
    beta: float = 1.0
    return alpha+edge_attributes['load']**beta

def method_2_weights( s: int, d: int, edge_attributes: dict):
    # define the weight function for the dijkstra or the bellman-ford algorithm.
    alpha: float = 10.0
    beta: float = 20.0
    return alpha+edge_attributes['load']**beta

In [13]:
_=nexu_it.optimize(2,10,method_1_weights, method_2_weights)

In [14]:
result_max_load = nexu_it.result_max_link_load
print("updated max remote link load: ", result_max_load)
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, result_max_load, max_local_link_load), "GBps")


updated max remote link load:  15.574218750000005
network_total_throughput =  741.9954853273135 GBps
